# --- Day 16: Proboscidea Volcanium ---

https://adventofcode.com/2022/day/16

## Get Input Data

In [29]:
import re

def get_data(filename):
    """Get input data for puzzle.
    
    Parameters
    ----------
    filename : str
        The name of the *.txt file in the inputs/ directory.
    
    Returns
    -------
    tuple(dict, dict)
        graph : dict
        flow_rates : dict
    """
    graph = {}
    flow_rates = {}

    re_str = r"Valve ([A-Z]{2}) .+=(\d+); .+ valves? (.+)"
    file_str = open(f"../inputs/{filename}.txt").read()

    for node, flow_rate, neighbors in re.findall(re_str, file_str):
        graph[node] = {}
        for neighbor in neighbors.split(", "):
            graph[node][neighbor] = 1
                
        flow_rates[node] = int(flow_rate)

    return graph, flow_rates

In [43]:
test_valves_graph, test_flow_rates = get_data("test_valves")
test_valves_graph

{'AA': {'DD': 1, 'II': 1, 'BB': 1},
 'BB': {'CC': 1, 'AA': 1},
 'CC': {'DD': 1, 'BB': 1},
 'DD': {'CC': 1, 'AA': 1, 'EE': 1},
 'EE': {'FF': 1, 'DD': 1},
 'FF': {'EE': 1, 'GG': 1},
 'GG': {'FF': 1, 'HH': 1},
 'HH': {'GG': 1},
 'II': {'AA': 1, 'JJ': 1},
 'JJ': {'II': 1}}

In [44]:
test_flow_rates

{'AA': 0,
 'BB': 13,
 'CC': 2,
 'DD': 20,
 'EE': 3,
 'FF': 0,
 'GG': 0,
 'HH': 22,
 'II': 0,
 'JJ': 21}

In [45]:
valves_graph, flow_rates = get_data("valves")

In [46]:
flow_rates

{'XB': 0,
 'BM': 0,
 'GC': 0,
 'RM': 0,
 'ZM': 5,
 'UH': 0,
 'GW': 0,
 'HN': 19,
 'VT': 0,
 'VI': 0,
 'YL': 12,
 'LA': 0,
 'CM': 0,
 'JI': 24,
 'ZD': 25,
 'VB': 0,
 'FO': 0,
 'JS': 0,
 'RI': 0,
 'XD': 14,
 'ES': 11,
 'WZ': 0,
 'HW': 0,
 'KE': 0,
 'IY': 22,
 'XW': 0,
 'BE': 0,
 'QY': 23,
 'MH': 0,
 'IT': 0,
 'DA': 0,
 'PM': 0,
 'WU': 0,
 'UL': 0,
 'SM': 13,
 'XC': 0,
 'OJ': 0,
 'SN': 0,
 'WH': 0,
 'UW': 0,
 'HM': 0,
 'AB': 0,
 'QR': 8,
 'UV': 0,
 'ZQ': 20,
 'ZJ': 0,
 'GL': 0,
 'CP': 0,
 'AM': 0,
 'PL': 17,
 'YE': 0,
 'PE': 4,
 'MY': 0,
 'QX': 0,
 'GI': 0,
 'WW': 0,
 'FF': 0,
 'LE': 6,
 'GU': 0,
 'AA': 0}

In [47]:
len([x for x in flow_rates.values() if int(x) > 0])

15

## Part 1
---

In [48]:
from collections import defaultdict
from itertools import product

def fw(graph):
    """Find the shortest path between all node pairs in a graph using the 
    Floyd-Warshall algorithm.

    Parameters
    ----------
    graph : dict of dicts
        Each key in outer dict is a node, and each value is another dict. Within
        the inner dict, each key is an adjacent/neighbor node to the outer dict
        key, and each value is the edge weight.

    Returns
    -------
    dist : dict
        Keys are (i, j) tuples, for all pairs of nodes i and j, and values are
        the shortest weighted distance between node i and node j in the graph.
    """
    # Create an empty default dictionary, with default values set at infinity.
    # Note that instead of an n x n matrix being returned, as is typical for 
    # most implementations of Floyd-Warshall, this will return a dictionary,
    # where the keys are an (i, j) tuple and the values are the min distance
    # between nodes i and j.
    dist = defaultdict(lambda: float("inf"))

    # Initialize the dist[(i, j)] values, where possible:
    # Diagonals will be set to 0s and direct neighbors will be given their
    # edge weights.
    for node, neighbors in graph.items():
        dist[(node, node)] = 0
        for neighbor, edge_weight in neighbors.items():
            dist[(node, neighbor)] = edge_weight
            dist[(neighbor, node)] = edge_weight

    # Update initialized distances
    # (use itertools.product for a aore efficient way to do for k: for i: for j:)
    for k, i, j in product(graph.keys(), repeat=3):
        dist[(i, j)] = min(dist[(i, j)], dist[(i, k)] + dist[(k, j)])

    return dict(dist)

In [49]:
fw(test_valves_graph)

{('AA', 'AA'): 0,
 ('AA', 'DD'): 1,
 ('DD', 'AA'): 1,
 ('AA', 'II'): 1,
 ('II', 'AA'): 1,
 ('AA', 'BB'): 1,
 ('BB', 'AA'): 1,
 ('BB', 'BB'): 0,
 ('BB', 'CC'): 1,
 ('CC', 'BB'): 1,
 ('CC', 'CC'): 0,
 ('CC', 'DD'): 1,
 ('DD', 'CC'): 1,
 ('DD', 'DD'): 0,
 ('DD', 'EE'): 1,
 ('EE', 'DD'): 1,
 ('EE', 'EE'): 0,
 ('EE', 'FF'): 1,
 ('FF', 'EE'): 1,
 ('FF', 'FF'): 0,
 ('FF', 'GG'): 1,
 ('GG', 'FF'): 1,
 ('GG', 'GG'): 0,
 ('GG', 'HH'): 1,
 ('HH', 'GG'): 1,
 ('HH', 'HH'): 0,
 ('II', 'II'): 0,
 ('II', 'JJ'): 1,
 ('JJ', 'II'): 1,
 ('JJ', 'JJ'): 0,
 ('AA', 'CC'): 2,
 ('AA', 'EE'): 2,
 ('AA', 'FF'): 3,
 ('AA', 'GG'): 4,
 ('AA', 'HH'): 5,
 ('AA', 'JJ'): 2,
 ('BB', 'DD'): 2,
 ('BB', 'EE'): 3,
 ('BB', 'FF'): 4,
 ('BB', 'GG'): 5,
 ('BB', 'HH'): 6,
 ('BB', 'II'): 2,
 ('BB', 'JJ'): 3,
 ('CC', 'AA'): 2,
 ('CC', 'EE'): 2,
 ('CC', 'FF'): 3,
 ('CC', 'GG'): 4,
 ('CC', 'HH'): 5,
 ('CC', 'II'): 3,
 ('CC', 'JJ'): 4,
 ('DD', 'BB'): 2,
 ('DD', 'FF'): 2,
 ('DD', 'GG'): 3,
 ('DD', 'HH'): 4,
 ('DD', 'II'): 2,
 ('DD', 'J

In [5]:
import numpy as np  # import this to access np.argmax() instead

In [75]:
def max_cum_pressure_release(valve_graph, flow_rates, mins_left=30, start_valve="AA"):
    flow_rates = flow_rates.copy()
    cum_pressure_released = 0
    valve = start_valve

    while mins_left > 0:
        print(f"== Minutes left {mins_left} ==")
        shortest_paths = dijkstra(valve_graph, valve)
        open_valves = [k for k in flow_rates if flow_rates[k] > 0]

        if len(open_valves) > 0:
            net_benefits = []

            for open_valve in open_valves:
                cost = shortest_paths[open_valve] + 1
                benefit = (mins_left - cost) * flow_rates[open_valve]
                net_benefits.append(benefit - cost)

                print(valve, open_valve, cost, benefit, benefit-cost)

            next_valve = open_valves[np.argmax(net_benefits)]

            valve = next_valve
            cost = shortest_paths[next_valve]

            cum_pressure_released += (mins_left - cost - 1) * flow_rates[next_valve]
            flow_rates[next_valve] = 0
            mins_left -= cost

        else:
            # Just chill -- all the valves are already open
            mins_left -= 1

        print(f"Current valve: {valve}")
        print(f"Cumulative pressure released: {cum_pressure_released}")

    return cum_pressure_released

### Run on Test Data

In [76]:
max_cum_pressure_release(test_valves_graph, test_flow_rates) # == 1651

== Minutes left 30 ==
AA BB 2 364 362
AA CC 3 54 51
AA DD 2 560 558
AA EE 3 81 78
AA HH 6 528 522
AA JJ 3 567 564
Current valve: JJ
Cumulative pressure released: 567
== Minutes left 28 ==
JJ BB 4 312 308
JJ CC 5 46 41
JJ DD 4 480 476
JJ EE 5 69 64
JJ HH 8 440 432
Current valve: DD
Cumulative pressure released: 1047
== Minutes left 25 ==
DD BB 3 286 283
DD CC 2 46 44
DD EE 2 69 67
DD HH 5 440 435
Current valve: HH
Cumulative pressure released: 1487
== Minutes left 21 ==
HH BB 7 182 175
HH CC 6 30 24
HH EE 4 51 47
Current valve: BB
Cumulative pressure released: 1669
== Minutes left 15 ==
BB CC 2 26 24
BB EE 4 33 29
Current valve: EE
Cumulative pressure released: 1702
== Minutes left 12 ==
EE CC 3 18 15
Current valve: CC
Cumulative pressure released: 1720
== Minutes left 10 ==
Current valve: CC
Cumulative pressure released: 1720
== Minutes left 9 ==
Current valve: CC
Cumulative pressure released: 1720
== Minutes left 8 ==
Current valve: CC
Cumulative pressure released: 1720
== Minutes lef

1720

### Run on Input Data

## Part 2
---

### Run on Test Data

### Run on Input Data